In [1]:
# import pandas as pd

# # Load datasets
# postings = pd.read_csv("Jobs Data/postings.csv")
# job_skills = pd.read_csv("Jobs Data/job_skills.csv")
# skills = pd.read_csv("Jobs Data/skills.csv")

In [2]:
# postings.columns

In [3]:
# job_skills.columns

In [4]:
# skills.columns

In [5]:

# # Merge job_skills with skill names
# job_skills_full = job_skills.merge(skills, on="skill_abr", how="left")


In [6]:
# skills.head()

In [7]:
# job_skills_full.head()

In [8]:
# # Aggregate skills per job
# skills_per_job = job_skills_full.groupby("job_id")["skill_name"].apply(lambda x: " ".join(x)).reset_index()


In [9]:
# skills_per_job.head()

In [10]:
# # Merge aggregated skills into postings
# jobs_combined = postings.merge(skills_per_job, on="job_id", how="left")

In [11]:
# jobs_combined.head(2)

In [12]:
# # Columns to drop
# cols_to_drop = [
#     'max_salary', 'pay_period', 'company_id', 'views', 'med_salary', 
#     'min_salary', 'formatted_work_type', 'applies', 'original_listed_time', 
#     'remote_allowed', 'job_posting_url', 'application_url', 'application_type', 
#     'expiry', 'closed_time', 'skills_desc', 'listed_time', 'posting_domain', 
#     'sponsored', 'work_type', 'currency', 'compensation_type', 'normalized_salary', 
#     'zip_code', 'fips', 'company_name'
# ]

# # Drop columns
# jobs_filtered = jobs_combined.drop(columns=cols_to_drop)

# # Save updated DataFrame to new CSV
# jobs_filtered.to_csv("jobs_filtered.csv", index=False)

# print("Updated jobs CSV saved as 'jobs_filtered.csv'")


In [13]:
import pandas as pd
Jobs_filtered = pd.read_csv("jobs_filtered.csv")


In [14]:
# Percentage of missing values per column
missing_percent = Jobs_filtered.isnull().sum() / len(Jobs_filtered) * 100
print(missing_percent)


job_id                         0.000000
title                          0.000000
description                    0.005652
location                       0.000000
formatted_experience_level    23.745852
skill_name                     1.415433
dtype: float64


In [15]:
Jobs_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123849 entries, 0 to 123848
Data columns (total 6 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   job_id                      123849 non-null  int64 
 1   title                       123849 non-null  object
 2   description                 123842 non-null  object
 3   location                    123849 non-null  object
 4   formatted_experience_level  94440 non-null   object
 5   skill_name                  122096 non-null  object
dtypes: int64(1), object(5)
memory usage: 5.7+ MB


In [16]:
Jobs_filtered['formatted_experience_level'].value_counts()

formatted_experience_level
Mid-Senior level    41489
Entry level         36708
Associate            9826
Director             3746
Internship           1449
Executive            1222
Name: count, dtype: int64

In [17]:
# fill small missing text columns with empty strings
Jobs_filtered['description'] = Jobs_filtered['description'].fillna('')
Jobs_filtered['skill_name'] = Jobs_filtered['skill_name'].fillna('')

# For experience level, you can:
# Option 1: Replace missing with 'Not Specified'
Jobs_filtered['formatted_experience_level'] = Jobs_filtered['formatted_experience_level'].fillna('Not Specified')


In [18]:
Jobs_filtered['combined_text'] = (
    Jobs_filtered['title'].astype(str) + ' ' +
    Jobs_filtered['description'].astype(str) + ' ' +
    Jobs_filtered['location'].astype(str) + ' ' +
    Jobs_filtered['formatted_experience_level'].astype(str) + ' ' +
    Jobs_filtered['skill_name'].astype(str)
)


In [19]:
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (only once)
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # 1. Lowercase
    text = text.lower()
    
    # 2. Remove punctuation / special characters
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. contractions 
    text = contractions.fix(text)   # <-- handle contractions here can’t to cannot
    
    # 4. Remove stopwords
    words = [word for word in text.split() if word not in stop_words]

    # 4. Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join words back to string
    return ' '.join(words)

# Apply to your column
Jobs_filtered['Document_Cleaned'] = Jobs_filtered['combined_text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to C:\Users\Faisal
[nltk_data]     Zamir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Faisal
[nltk_data]     Zamir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(Jobs_filtered["Document_Cleaned"])  # return sparse matrix (SciPy)


In [21]:
# Get feature names (vocabulary)
words = vectorizer.get_feature_names_out()

print("TF-IDF shape:", tfidf_matrix.shape)
print(words)

TF-IDF shape: (123849, 5000)
['aa' 'aaa' 'ab' ... 'zip' 'zone' 'zoom']


In [23]:
import pandas as pd

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out() 

# Convert to DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# See top rows
tfidf_df.head()


,aa,aaa,ab,aba,abide,ability,able,abreast,absence,abuse,...,yield,york,young,youth,youtube,yr,zero,zip,zone,zoom
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.022639,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.058318,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.265185,0.0,0.0,0.0,0.0


In [24]:
user_data = pd.read_csv("user_profiles.csv")
user_data.head()

,user_id,name,experience_level,job_title,skills,preferred_industry,location,years_experience,education_level
0,1,Faisal Zamir,Mid,Machine Learning Engineer,"Python, Scikit-learn, TensorFlow, NLP",Information Technology,"Islamabad, PK",3,Masters
1,2,Sara Ahmed,Entry,Web Developer,"HTML, CSS, JavaScript, React, PHP",Software Development,"Lahore, PK",1,Bachelors
2,3,Ali Khan,Senior,Data Scientist,"Python, Pandas, SQL, ML, Statistics",AI Research,"Karachi, PK",6,Masters
3,4,Ayesha Noor,Mid,Android Developer,"Java, Kotlin, Android Studio, Firebase",Mobile Development,"Rawalpindi, PK",4,Bachelors
4,5,Hassan Raza,Entry,Data Analyst,"Excel, SQL, PowerBI, Python",Business Analytics,"Faisalabad, PK",2,Bachelors


In [25]:
# Convert numeric experience into text
user_data["experience_text"] = user_data["years_experience"].astype(str) + " years experience"

# Combine all user info into one text column
user_data["profile_text"] = (
    user_data["job_title"].fillna('') + " " +
    user_data["skills"].fillna('') + " " +
    user_data["experience_level"].fillna('') + " " +
    user_data["experience_text"].fillna('') + " " +
    user_data["preferred_industry"].fillna('') + " " +
    user_data["education_level"].fillna('') + " " +
    user_data["location"].fillna('')
)
user_data['profile_text']

0    Machine Learning Engineer Python, Scikit-learn...
1    Web Developer HTML, CSS, JavaScript, React, PH...
2    Data Scientist Python, Pandas, SQL, ML, Statis...
3    Android Developer Java, Kotlin, Android Studio...
4    Data Analyst Excel, SQL, PowerBI, Python Entry...
5    Frontend Developer JavaScript, React, Vue, HTM...
6    DevOps Engineer AWS, Docker, Kubernetes, CI/CD...
7    QA Tester Selenium, Python, Manual Testing, Au...
8    Backend Developer Python, Django, REST API, SQ...
9    Business Analyst Excel, SQL, PowerBI, Data Ana...
Name: profile_text, dtype: object

In [26]:
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download NLTK resources (only once)
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    # 1. Lowercase
    text = text.lower()
    
    # 2. Remove punctuation / special characters
    text = re.sub(r'[^a-z\s]', '', text)
    
    # 3. contractions 
    text = contractions.fix(text)   # <-- handle contractions here can’t to cannot
    
    # 4. Remove stopwords
    words = [word for word in text.split() if word not in stop_words]

    # 4. Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]
    
    # Join words back to string
    return ' '.join(words)

# Apply to your column
user_data['Profile_Cleaned'] = user_data['profile_text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to C:\Users\Faisal
[nltk_data]     Zamir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Faisal
[nltk_data]     Zamir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
user_tfidf = vectorizer.transform(user_data["Profile_Cleaned"]) # return sparse matrix (SciPy)


In [ ]:
# There are two vector table : Document_Cleaned and Profile Cleaned (user_tfidf)

In [28]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim = cosine_similarity(user_tfidf, tfidf_matrix)


In [29]:
print(cosine_sim.shape)
# Convert to DataFrame
similarity_df = pd.DataFrame(
    cosine_sim,
    index=user_data["name"],         # user names as rows
    columns=Jobs_filtered["title"]   # job titles as columns
)

# Show first few users vs jobs
similarity_df.head()


(10, 123849)


title,Marketing Coordinator,Mental Health Therapist/Counselor,Assitant Restaurant Manager,Senior Elder Law / Trusts and Estates Associate Attorney,Service Technician,Economic Development and Planning Intern,Producer,Building Engineer,Respiratory Therapist,Worship Leader,...,Catering Event Manager,Phlebotomist - Float,Senior Frontend/App Developer,"Account Manager, Client Success",Quality Engineer,Title IX/Investigations Attorney,"Staff Software Engineer, ML Serving Platform","Account Executive, Oregon/Washington",Business Development Manager,Marketing Social Media Specialist
name,,,,,,,,,,,,,,,,,,,,,
Faisal Zamir,0.007345,0.008328,0.006852,0.006851,0.033168,0.004532,0.034806,0.059167,0.013007,0.014559,...,0.007406,0.010381,0.033450,0.017473,0.050867,0.008319,0.062804,0.019499,0.000000,0.015573
Sara Ahmed,0.005640,0.003302,0.005262,0.005261,0.004509,0.023719,0.003151,0.006593,0.004346,0.001979,...,0.005687,0.015124,0.255885,0.014386,0.006472,0.013376,0.009859,0.020170,0.011744,0.010083
Ali Khan,0.005558,0.000000,0.005185,0.024232,0.004443,0.020298,0.078175,0.006497,0.002924,0.001951,...,0.005604,0.007856,0.034346,0.014417,0.006378,0.006296,0.389872,0.014570,0.004479,0.010745
Ayesha Noor,0.004330,0.002535,0.004040,0.004039,0.003462,0.018209,0.002419,0.005062,0.003337,0.001520,...,0.004366,0.006120,0.297183,0.057848,0.004969,0.010269,0.002443,0.011702,0.009016,0.004364
Hassan Raza,0.007814,0.000000,0.007289,0.007289,0.019454,0.017261,0.004365,0.011747,0.004111,0.002742,...,0.007879,0.020953,0.031277,0.051424,0.008967,0.017378,0.026981,0.026247,0.060040,0.021217


In [30]:
for i, user in enumerate(similarity_df.index):
    top_indices = similarity_df.iloc[i].sort_values(ascending=False).head(5).index
    print(f"\nTop 5 jobs for {user}:")
    print(top_indices)



Top 5 jobs for Faisal Zamir:
Index(['Developer', 'Senior Machine Learning Engineer',
       'Machine Learning Engineer - Remote',
       'Machine Learning Engineer - Remote',
       'Machine Learning Engineer - Remote'],
      dtype='object', name='title')

Top 5 jobs for Sara Ahmed:
Index(['React Developer', 'Vue Developer', 'Javascript Developer',
       'Senior Front End Developer with Vue JS experience',
       'React Developer (Only W2)'],
      dtype='object', name='title')

Top 5 jobs for Ali Khan:
Index(['Analyst', 'Senior Data Scientist',
       'Sr. Data Scientist - IoT and Manufacturing',
       'Senior Machine Learning Engineer', 'Python Developer'],
      dtype='object', name='title')

Top 5 jobs for Ayesha Noor:
Index(['Android Developer', 'Android Developer', 'Lead Android Developer ',
       'Senior Android Developer - Remote / Telecommute', 'Android Developer'],
      dtype='object', name='title')

Top 5 jobs for Hassan Raza:
Index(['Data Analyst',
       'Python Deve

In [31]:
# Example: Select Faisal Zamir's similarity scores
user_similarities = similarity_df.head().loc["Faisal Zamir"]


In [32]:
top_5_jobs = user_similarities.sort_values(ascending=False).head(5)


In [33]:
top_5_df = top_5_jobs.reset_index()
top_5_df.columns = ['Job Title', 'Similarity Score']
print(top_5_df)


                            Job Title  Similarity Score
0                           Developer          0.460730
1    Senior Machine Learning Engineer          0.440024
2  Machine Learning Engineer - Remote          0.437329
3  Machine Learning Engineer - Remote          0.437287
4  Machine Learning Engineer - Remote          0.435892


In [34]:
import joblib
import pandas as pd

# Save the essential components
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")
joblib.dump(tfidf_matrix, "jobs_tfidf_matrix.pkl")

# Save the job data for reference
Jobs_filtered[['title', 'description', 'location', 'formatted_experience_level', 'skill_name']].to_pickle("jobs_data.pkl")

In [35]:
# Load all saved components
vectorizer = joblib.load("tfidf_vectorizer.pkl")
jobs_tfidf_matrix = joblib.load("jobs_tfidf_matrix.pkl")
jobs_data = pd.read_pickle("jobs_data.pkl")

In [36]:
# Example new unseen user data
new_user = pd.DataFrame({
    "name": ["Ali Raza"],
    "job_title": ["Data Analyst"],
    "skills": ["Python, SQL, Excel, Power BI"],
    "experience_level": ["Entry"],
    "years_experience": [1],
    "preferred_industry": ["Information Technology"],
    "location": ["Lahore, PK"],
    "education_level": ["Bachelors"]
})

# Combine into one text
new_user["experience_text"] = new_user["years_experience"].astype(str) + " years experience"
new_user["profile_text"] = (
    new_user["job_title"] + " " +
    new_user["skills"] + " " +
    new_user["experience_level"] + " " +
    new_user["experience_text"] + " " +
    new_user["preferred_industry"] + " " +
    new_user["education_level"] + " " +
    new_user["location"]
)


In [37]:
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    text = contractions.fix(text)
    words = [word for word in text.split() if word not in stop_words]
    words = [lemmatizer.lemmatize(word) for word in words]
    return ' '.join(words)

new_user["Profile_Cleaned"] = new_user["profile_text"].apply(preprocess_text)


In [38]:
# Convert using existing TF-IDF vectorizer
new_user_tfidf = vectorizer.transform(new_user["Profile_Cleaned"])

# Calculate cosine similarity with job TF-IDF matrix
cosine_sim_new = cosine_similarity(new_user_tfidf, tfidf_matrix)

# Get top 5 matching jobs
similarity_series = pd.Series(cosine_sim_new[0], index=Jobs_filtered["title"])
top_5 = similarity_series.sort_values(ascending=False).head(5)

# Display
print(f"\nTop job recommendations for {new_user['name'][0]}:\n")
print(top_5)



Top job recommendations for Ali Raza:

title
Data Analyst - Power BI & SQL    0.600978
BI Analyst                       0.599566
Power BI Architect               0.570967
Power BI Developer               0.562427
Senior PowerBi Designer          0.553911
dtype: float64


In [ ]:
# def recommend_jobs(user_profile_dict):
#     # Load pickled models
#     with open("tfidf_vectorizer.pkl", "rb") as f:
#         vectorizer = pickle.load(f)
#     with open("tfidf_matrix.pkl", "rb") as f:
#         tfidf_matrix = pickle.load(f)
#     with open("jobs_filtered.pkl", "rb") as f:
#         Jobs_filtered = pickle.load(f)

#     # Create user DataFrame
#     user_df = pd.DataFrame([user_profile_dict])
#     user_df["experience_text"] = user_df["years_experience"].astype(str) + " years experience"
#     user_df["profile_text"] = (
#         user_df["job_title"] + " " +
#         user_df["skills"] + " " +
#         user_df["experience_level"] + " " +
#         user_df["experience_text"] + " " +
#         user_df["preferred_industry"] + " " +
#         user_df["education_level"] + " " +
#         user_df["location"]
#     )

#     user_df["Profile_Cleaned"] = user_df["profile_text"].apply(preprocess_text)

#     # Vectorize and compute similarity
#     user_vec = vectorizer.transform(user_df["Profile_Cleaned"])
#     cosine_sim = cosine_similarity(user_vec, tfidf_matrix)[0]

#     # Find top 5 jobs
#     top_jobs = pd.Series(cosine_sim, index=Jobs_filtered["title"]).sort_values(ascending=False).head(5)
#     return top_jobs


In [ ]:
# recommend_jobs({
#     "name": "Ali Raza",
#     "job_title": "Data Analyst",
#     "skills": "Python, SQL, Excel, Power BI",
#     "experience_level": "Entry",
#     "years_experience": 1,
#     "preferred_industry": "Information Technology",
#     "location": "Lahore, PK",
#     "education_level": "Bachelors"
# })
